In [1]:
import pandas as pd
import numpy as np

def generate_points(size, min, max):
    points = np.random.normal(min, max, (size, 3))
    return np.ndarray.tolist(points)

# Randomly generates points with coordinates values ranging from -10 to 10
positions = generate_points(100000, -10, 10)
positions[:10]

#Uncomment the lines below to use the positions file instead
df_points = pd.read_table("data/positions.xyz", delim_whitespace=True, names=['x', 'y', 'z'])
print(df_points)
positions = np.ndarray.tolist(df_points.to_numpy().reshape(-1, 3))

              x         y         z
0     -3.719171  1.441667  2.574435
1      3.371187 -1.834861 -5.104661
2      2.861366 -3.024369 -5.513213
3      2.837314 -2.033112 -5.512642
4      0.819776 -0.647488 -0.632704
...         ...       ...       ...
16119 -4.331544 -1.183650  4.431044
16120 -4.240115  0.342916  4.239984
16121 -3.959691 -1.034959  4.024511
16122 -3.749135  0.937679  4.394070
16123 -4.371996  1.251320  3.958347

[16124 rows x 3 columns]


In [2]:
def binary_search(array, lo, hi, key, last=False):
    result = -1
    while lo < hi:
        mid = (hi + lo) // 2
        comparator = key(array[mid])

        if comparator == 0:
            result = mid
            if last:
                lo = mid + 1
            else:
                hi = mid
        elif comparator < 0:
            lo = mid + 1
        else:
            hi = mid
    
    return result

def distance(p1, p2):
    d = 0
    for i in range(len(p1)):
        d += (p1[i] - p2[i])**2
    return d**0.5

def brute_force_(points, lo, hi, min_d, pairs, lo1, hi1):
    local_pairs = []
    for i in range(lo, hi):
        for j in range(lo1, hi1):
            p1 = points[i]
            p2 = points[j]
            d = distance(p1, p2)
            if d < min_d and (p1, p2) not in local_pairs and (p2, p1) not in local_pairs:
                local_pairs.append((p1, p2))
    pairs += local_pairs

def brute_force(points, lo, hi, min_d, pairs):
    brute_force_(points, lo, hi, min_d, pairs, lo, hi)

def close_to_mid(p, mid_point, min_d, dim):
    d = p[dim] - mid_point[dim]
    if abs(d) <= min_d:
        return 0
    return d

def find_close_pairs(points, lo, hi, min_d, cutoff, dim_depth, verbose=[]):
    pairs = []
    find_close_pairs_recursion(points, lo, hi, min_d, cutoff, dim_depth, verbose, 0, pairs)
    return pairs

i = 0
def find_close_pairs_recursion(points, lo, hi, min_d, cutoff, dim_depth, verbose, dim, pairs):
    global i
    i += 1
    size = hi - lo
    if size <= cutoff:
        brute_force(points, lo, hi, min_d, pairs)
        return

    mid = (lo + hi) // 2
    mid_point = points[mid]

    lo_close_to_mid = close_to_mid(points[lo], mid_point, min_d, dim) == 0
    hi_close_to_mid = close_to_mid(points[hi-1], mid_point, min_d, dim) == 0

    if (lo_close_to_mid or hi_close_to_mid) and dim_depth - dim > 1:
        local_points = sorted(points[lo:hi], key=lambda p: p[dim + 1])
        if "dim" in verbose:
            print("next dim!", "dim", dim, "to", "dim", dim+1)
        find_close_pairs_recursion(local_points, 0, size, min_d, cutoff, dim_depth, verbose, dim+1, pairs)
        return

    mid_lo = lo
    if not lo_close_to_mid:
        mid_lo = binary_search(points, lo, mid, lambda p: close_to_mid(p, mid_point, min_d, dim), False)
    elif "close" in verbose:
        print("too close lo", mid-lo, "dim", dim)

    # mid_lo1 = mid - 1
    # while mid_lo1 > lo and close_to_mid(points[mid_lo1], mid_point, min_d, dim) == 0:
    #     mid_lo1 -= 1
        
    mid_hi = hi
    if not hi_close_to_mid:
        mid_hi = binary_search(points, mid, hi, lambda p: close_to_mid(p, mid_point, min_d, dim), True) + 1
    elif "close" in verbose:
        print("too close hi", hi-mid, "dim", dim)

    # mid_hi1 = mid + 1
    # while mid_hi1 < hi and close_to_mid(points[mid_hi1], mid_point, min_d, dim) == 0:
    #     mid_hi1 += 1

    # if mid_hi != mid_hi1:
    #     print("Difference mid_hi!", mid_hi, mid_hi1, hi)
    # if mid_lo != mid_lo1:
    #     print("Difference mid_lo!", mid_lo, mid_lo1, lo)

    brute_force_(points, mid_lo, mid, min_d, pairs, mid, mid_hi)
    find_close_pairs_recursion(points, lo, mid, min_d, cutoff, dim_depth, verbose, dim, pairs)
    find_close_pairs_recursion(points, mid, hi, min_d, cutoff, dim_depth, verbose, dim, pairs)

In [3]:
import time

positions = positions

# reduce the amount of points
divide_positions_by = 1

size = len(positions)//divide_positions_by
min_d = 0.05
cutoff = 20
print(f"Parameters: min_d = {min_d}, cutoff = {cutoff}, Points: {size}")
i = 0
for dim_depth in [1,2,3]:
    start_time = time.time()
    positions = sorted(positions, key=lambda p: p[0])

    q_pairs = find_close_pairs(positions, 0, size, min_d, cutoff, dim_depth, verbose=[""])
    total_time = time.time() - start_time
    print(f"Finding pairs with dim_depth = {dim_depth}: {round(total_time, 4)}s. Pairs: {len(q_pairs)}")
    print("i:", i)
    i = 0

Parameters: min_d = 0.05, cutoff = 20, Points: 16124
Finding pairs with dim_depth = 1: 9.1305s. Pairs: 35479
i: 2047
Finding pairs with dim_depth = 2: 5.0263s. Pairs: 35479
i: 2161
Finding pairs with dim_depth = 3: 5.1747s. Pairs: 35479
i: 2173


In [4]:
bf_pairs = []
brute_force(positions, 0, size, min_d, bf_pairs)

print("Brute force:", len(bf_pairs))

KeyboardInterrupt: 